In [1]:
import folium
import numpy as np
import pandas as pd
from pulp import LpMaximize, LpProblem, LpVariable, lpSum

# Generate synthetic data for 25 venues
data = pd.DataFrame({
    'Name': ['Venue' + str(i) for i in range(25)],
    'Latitude': np.random.uniform(48.830, 48.880, 25),
    'Longitude': np.random.uniform(2.260, 2.410, 25),
    'Satisfaction': np.random.randint(5, 10, 25),
    'EntranceFee': np.random.uniform(5, 20, 25)
})

# Model to optimize the visitation schedule
model = LpProblem("Art_Lover_Tour_Optimization", LpMaximize)
visits = LpVariable.dicts("Visit", range(25), cat='Binary')  # 0 or 1 for visiting each venue

# Objective: Maximize satisfaction
model += lpSum([visits[i] * data.loc[i, 'Satisfaction'] for i in range(25)])

# Constraints
model += lpSum([visits[i] * data.loc[i, 'EntranceFee'] for i in range(25)]) <= 50  # Budget constraint
model += lpSum([visits[i] for i in range(25)]) == 10  # Total venues to visit

# Solve the model
model.solve()

# Selecting indices for first and second day
selected_indices = [i for i in range(25) if visits[i].varValue == 1]
day1, day2 = selected_indices[:5], selected_indices[5:10]

# Create a Folium map centered on Paris
map = folium.Map(location=[48.8566, 2.3522], zoom_start=12, tiles='OpenStreetMap')

# Adding markers to the map
for idx in day1:
    folium.Marker(
        location=[data.loc[idx, 'Latitude'], data.loc[idx, 'Longitude']],
        popup=data.loc[idx, 'Name'],
        icon=folium.Icon(color='red', icon='info-sign')
    ).add_to(map)

for idx in day2:
    folium.Marker(
        location=[data.loc[idx, 'Latitude'], data.loc[idx, 'Longitude']],
        popup=data.loc[idx, 'Name'],
        icon=folium.Icon(color='orange', icon='info-sign')
    ).add_to(map)

# Display the map
map.save('Optimal_Art_Tour.html')
map


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/anaconda3/envs/OR/lib/python3.12/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/9f/pv1nlhw528d_5zttzbkb_h5m0000gn/T/d35c04d768974a8d99921e34019b6808-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/9f/pv1nlhw528d_5zttzbkb_h5m0000gn/T/d35c04d768974a8d99921e34019b6808-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 7 COLUMNS
At line 133 RHS
At line 136 BOUNDS
At line 162 ENDATA
Problem MODEL has 2 rows, 25 columns and 50 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Problem is infeasible - 0.00 seconds
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.02

